In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import load_model
import os
import numpy as np
import cv2
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation,Conv2DTranspose
from keras.layers import BatchNormalization
from keras.layers import UpSampling2D


In [2]:
train_data = np.load('train_data_112.npy')
test_data = np.load('test_data_112.npy')
train_images = train_data[:, :-12].reshape(-1, 112, 112, 3)
test_images = test_data[:, :-12].reshape(-1, 112, 112, 3)



In [3]:
train_X = train_images / 255.0
test_X = test_images / 255.0
input_shape = (112, 112, 3)
num_classes = 12

target_data = np.expand_dims(train_data[:, -12:], axis=(1, 2))
test_y = np.expand_dims(test_data[:, -12:], axis=(1, 2))
train_y = np.tile(target_data.reshape(-1, 1, 1, num_classes), (1, 112, 112, 1))
test_y = np.tile(test_y.reshape(-1, 1, 1, num_classes), (1, 112, 112, 1))
print(np.shape(test_X))
print(np.shape(train_y))
print(np.shape(test_y))

(3103, 112, 112, 3)
(12412, 112, 112, 12)
(3103, 112, 112, 12)


In [4]:
from tensorflow.keras import backend as K

def custom_precision(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def custom_recall(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def custom_f1_score(y_true, y_pred):
    precision = custom_precision(y_true, y_pred)
    recall = custom_recall(y_true, y_pred)
    f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_score

In [5]:



with tf.device('/GPU:0'):
    model = Sequential()
    # Encoder
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))  # 56x56x32

    model.add(Conv2D(128, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))  # 28x28x256

    model.add(Conv2D(512, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))  #14x14x256

    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))  # 7x7x256

    model.add(Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same'))  # 14x14x512
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same'))  # 28x28x256
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))  # 56x56x128
    model.add(Activation('relu'))
    model.add(Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same'))  # 112x112x64
    model.add(Activation('relu'))
    model.add(Conv2D(num_classes, (1, 1), activation='softmax', padding='valid'))


    model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy',
                       custom_f1_score,
                       custom_precision,
                       custom_recall])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 112, 112, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 64)     256       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 112, 112, 64)      0

In [6]:
#
with tf.device('/GPU:0'):
    model.fit(train_X, train_y, epochs=20, batch_size=32, validation_data=(test_X, test_y))
model.save('fcn_model.h5')
#

test_loss, test_acc = model.evaluate(test_X, test_y)
print('Test accuracy:', test_acc)

Epoch 1/20
388/388 [==============================] - 33s 62ms/step - loss: 1.9457 - accuracy: 0.3785 - custom_f1_score: 0.3037 - custom_precision: 0.7089 - custom_recall: 0.2026 - val_loss: 1.8444 - val_accuracy: 0.3928 - val_custom_f1_score: 0.3898 - val_custom_precision: 0.6609 - val_custom_recall: 0.2793
Epoch 2/20
388/388 [==============================] - 21s 53ms/step - loss: 1.6975 - accuracy: 0.4410 - custom_f1_score: 0.3870 - custom_precision: 0.7385 - custom_recall: 0.2654 - val_loss: 1.8506 - val_accuracy: 0.4553 - val_custom_f1_score: 0.4477 - val_custom_precision: 0.6373 - val_custom_recall: 0.3475
Epoch 3/20
388/388 [==============================] - 21s 53ms/step - loss: 1.5599 - accuracy: 0.4760 - custom_f1_score: 0.4163 - custom_precision: 0.7539 - custom_recall: 0.2903 - val_loss: 1.6133 - val_accuracy: 0.4775 - val_custom_f1_score: 0.3796 - val_custom_precision: 0.7533 - val_custom_recall: 0.2562
Epoch 4/20
388/388 [==============================] - 21s 54ms/step - 

ValueError: too many values to unpack (expected 2)

In [9]:
model = load_model('h5 files/fcn_model.h5')
test_loss, test_acc= model.evaluate(test_X, test_y)
print('Test accuracy:', test_acc)

97/97 [==============================] - 2s 17ms/step - loss: 0.8222 - accuracy: 0.7518
Test accuracy: 0.7518138885498047
